In [3]:
import os
import PIL
import pathlib
import PIL.Image
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm as notebook_tqdm

Data Exploration

In [ ]:
data_dir = pathlib.Path(archive).with_suffix('')

In [ ]:
data_dir = 

list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)

Feature Engineering

Supervised Learning

Unsupervised or Semisupervised Learning

Model Deployment/Evaluation